Klasifikasi cats vs dogs menggunakan SVM

In [1]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


Baca Data

In [2]:
local = 'cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [3]:

# Load and preprocess the data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

image_size = (150, 150)  # Define the desired image size

for img_path in os.listdir(train_cats_dir):
    image = cv2.imread(os.path.join(train_cats_dir, img_path))
    if image is not None:
        image = cv2.resize(image, image_size)  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        image = image / 255.0  # Normalize pixel values (if needed)
        image = image.flatten()  # Flatten the image into a 1D array
        image_paths.append(image)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(train_dogs_dir):
    image = cv2.imread(os.path.join(train_dogs_dir, img_path))
    if image is not None:
        image = cv2.resize(image, image_size)  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        image = image / 255.0  # Normalize pixel values (if needed)
        image = image.flatten()  # Flatten the image into a 1D array
        image_paths.append(image)
        labels.append(1)  # 1 represents dogs

# Convert the image_paths and labels lists to NumPy arrays
X = np.array(image_paths)
y = np.array(labels)

# Split data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid to search
param_grid = {
    'C': [0.1, 1, 10],           # Regularization parameter
    'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],  # Kernel type (linear or radial basis function)
}

# Create an SVM classifier
svm_classifier = svm.SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm_classifier, param_grid, cv=7)  # You can adjust the number of cross-validation folds

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Make predictions using the best model
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)

# Calculate metrics for the validation set
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Precision: {precision * 100:.2f}%")
print(f"Test Recall: {recall * 100:.2f}%")
print(f"Test F1: {recall * 100:.2f}%")

Best Parameters: {'C': 1, 'kernel': 'rbf'}
Test Accuracy: 61.75%
Test Precision: 61.32%
Test Recall: 64.68%
Test F1: 64.68%


In [4]:
# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Load and preprocess the data
image_paths = []  # List of image file paths
labels = []  # List of corresponding labels (0 for cats, 1 for dogs)

image_size = (150, 150)  # Define the desired image size

for img_path in os.listdir(validation_cats_dir):
    image = cv2.imread(os.path.join(validation_cats_dir, img_path))
    if image is not None:
        image = cv2.resize(image, image_size)  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        image = image / 255.0  # Normalize pixel values
        image = image.flatten()  # Flatten the image into a 1D array
        image_paths.append(image)
        labels.append(0)  # 0 represents cats

for img_path in os.listdir(validation_dogs_dir):
    image = cv2.imread(os.path.join(validation_dogs_dir, img_path))
    if image is not None:
        image = cv2.resize(image, image_size)  # Resize the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        image = image / 255.0  # Normalize pixel values
        image = image.flatten()  # Flatten the image into a 1D array
        image_paths.append(image)
        labels.append(1)  # 1 represents dogs

# Convert the image_paths and labels lists to NumPy arrays
X_validation = np.array(image_paths)
y_validation = np.array(labels)

# Use your trained SVM model to make predictions on the validation data
y_validation_pred = best_model.predict(X_validation)

# Calculate metrics for the validation data
accuracy_validation = accuracy_score(y_validation, y_validation_pred)
precision_validation = precision_score(y_validation, y_validation_pred)
recall_validation = recall_score(y_validation, y_validation_pred)
f1_validation = f1_score(y_validation, y_validation_pred)

print(f"Validation Accuracy: {accuracy_validation * 100:.2f}%")
print(f"Validation Precision: {precision_validation * 100:.2f}%")
print(f"Validation Recall: {recall_validation * 100:.2f}%")
print(f"Validation F1: {recall_validation * 100:.2f}%")

Validation Accuracy: 60.00%
Validation Precision: 59.62%
Validation Recall: 62.00%
Validation F1: 62.00%
